In [46]:
#wandb.finish()
#!pip install --upgrade pip
#!pip install "transformers==4.30.2" 
#!pip install "datasets[s3]==2.13.0" 
#!pip install sagemaker --upgrade 
#!pip install "transformers==4.30.2" --upgrade
#!pip3 install git+https://github.com/huggingface/transformers
#!pip install torch

In [116]:
from getpass import getpass
import os
from datetime import datetime
import sagemaker
import boto3
import torch
##import wandb
from huggingface_hub import login
from transformers import LlamaTokenizer
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk
import time
from sagemaker.huggingface import HuggingFace
from huggingface_hub import HfFolder
import wandb

hf_token=None
wandb_token=None

#wandb.login()

#wandb.finish()
#wandb.init(
#    job_type = 'training',
#    dir = './',
#    project='sagemaker_training_test',
#    entity ='ft-llmmm'
#)

In [117]:
hf_token = getpass('input hf token')
wandb_token = getpass('input wandb token')

In [118]:
sess = sagemaker.Session()
sagemaker_session_bucket='llms-hf'

iam = boto3.client('iam')
role = iam.get_role(RoleName='DataScientist')['Role']['Arn']

#sagemaker_session_bucket = sess.default_bucket()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

artifact_dir='artifacts/combined_dataset:v1'

In [119]:
print(f"Logging into the Hugging Face Hub with token {hf_token[:10]}...")
login(token=hf_token)

Logging into the Hugging Face Hub with token hf_VLXemeC...
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/davidmeltzer/.cache/huggingface/token
Login successful


In [123]:
wandb.login(key=wandb_token)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/davidmeltzer/.netrc


True

In [ ]:
#now = datetime.now()
#time_stamp = now.strftime("%m.%d.%y-%H.%M.%S")
#with wandb.init(project='SFT_training_DM',
#                entity='ft-llmmm',
#                job_type='download_data',
#                name=f'download_combined_data_{time_stamp}') as run:
#
#    artifact = run.use_artifact('ft-llmmm/ELI5_analysis/combined_dataset:v1', type='dataset')
#    artifact_dir = artifact.download()

In [ ]:
model_id = "meta-llama/Llama-2-7b-hf" # sharded weights
tokenizer = AutoTokenizer.from_pretrained(model_id,use_auth_token=hf_token)
tokenizer.pad_token = tokenizer.eos_token

model_name = model_id.split('/')[-1]



ds_QA_SFT = load_from_disk(f'../{artifact_dir}')
ds_wiki = ds_QA_SFT.filter(lambda x:x['source']=='simple_wiki')
ds_ELI5 = ds_QA_SFT.filter(lambda x:x['source']=='ELI5')

ds_name='wiki'

if ds_name == 'combined':
    ds = ds_QA_SFT
elif ds_name == 'wiki':
    ds = ds_wiki
elif ds_name == 'ELI5':
    ds = ds_ELI5
else:
    raise ValueError("Input valid dataset name, 'combined','ELI5', or 'wiki'")

training_input_path = f's3://{sagemaker_session_bucket}/{ds_name}'
#ds.save_to_disk(training_input_path)

# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

run_name = f'{model_name}_{ds_name}_qlora'
run_name += f'__{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

In [136]:
job_name = f'huggingface-qlora-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

hyperparameters ={
  'model_id': model_id,
  'dataset_path': '/opt/ml/input/data/training',
  'epochs': 1,
  'per_device_train_batch_size': 8,
  'lr': 2e-4,
  'hf_token': hf_token,
  'wandb_token': wandb_token,
  'merge_weights': False,
  'per_device_eval_batch_size': 8,
  'gradient_accumulation_steps': 6,
  'max_seq_length': 512,
  'max_steps': 1,
  'entity': 'ft-llmmm',
  'project_name': 'SFT_training_dm',
  'run_name': run_name
}


In [137]:
# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_clm.py',      # train script
    source_dir           = './scripts',         # directory which includes all the files needed for training
    instance_type        = 'ml.g5.4xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.28',            # the transformers version used in the training job
    pytorch_version      = '2.0',             # the pytorch_version version used in the training job
    py_version           = 'py310',           # the python version used in the training job
    hyperparameters      =  hyperparameters,  # the hyperparameters passed to the training job
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache" }, # set env variable to cache models in /tmp
)

In [138]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-qlora-2023-08-10-15-46-59-2023-08-10-19-47-02-683


2023-08-10 19:47:03 Starting - Starting the training job...
2023-08-10 19:47:19 Starting - Preparing the instances for training......
2023-08-10 19:48:30 Downloading - Downloading input data...
2023-08-10 19:48:51 Failed - Training job failed
..

UnexpectedStatusException: Error for Training job huggingface-qlora-2023-08-10-15-46-59-2023-08-10-19-47-02-683: Failed. Reason: ClientError: Data download failed:Failed to download data. 403 Forbidden (403): Forbidden